# Scraping beritajatim.com

## Import Library

**requests** : digunakan untuk membuat permintaan HTTP ke situs web

**BeautifulSoup** : digunakan untuk mengurai konten HTML



In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Scraping

Fungsi ini melakukan iterasi melalui setiap halaman berita dalam rentang yang ditentukan oleh hal.
Pada setiap halaman, fungsi mengambil konten HTML menggunakan requests.get() dari tautan yang dibentuk berdasarkan nomor halaman.
Jika respons dari permintaan adalah 200 (artinya halaman berhasil dimuat), konten halaman di-parse menggunakan BeautifulSoup untuk ekstraksi informasi.
Fungsi mencari blok konten berita dengan class widget berita-list.
Di dalam blok konten tersebut, ia mencari semua elemen berita dengan class berita.
Untuk setiap artikel berita, fungsi mengekstrak judul dari elemen h3 dan tautan dari elemen a di dalamnya.
Selanjutnya, fungsi melakukan permintaan HTTP GET ke tautan artikel untuk mengambil konten dari setiap artikel secara individu.
Jika konten artikel berhasil diambil (status respons 200), fungsi mengekstrak konten dari elemen dengan class entry-content dan mengumpulkannya.
Setelah mengumpulkan judul, konten, dan kategori dari setiap artikel, informasi tersebut dimasukkan ke dalam list data.
Akhirnya, fungsi mengembalikan list data yang berisi informasi yang telah berhasil di-scrape dari halaman-halaman berita.










In [30]:
def scrap_beritajatim(link, hal, kategori):
    category = kategori
    data = []
    base_url = link
    for page in range(1, hal + 1):
        url = f'{base_url}page/{page}/'
        # print(f"Scraping page: {url}")
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            blok = soup.find('div', class_='widget berita-list')
            articles = blok.find_all('div', class_='berita')
            for article in articles:
                link = article.find('a')['href']
                # print(f"link: {link}")
                title = article.find('h3').text.strip()
                # print(f"Judul: {title}
            content_response = requests.get(link)
            if content_response.status_code == 200:
                content_soup = BeautifulSoup(content_response.content, "html.parser")
                article_content = content_soup.find('div', class_='entry-content')
                if article_content:
                    content_text = ""
                    for paragraph in article_content.find_all(['p', 'h2', 'h3', 'h4', 'h5']):
                        content_text += paragraph.get_text(strip=True).replace('\n', '').replace('ADVERTISEMENT','').replace('SCROLL TO RESUME CONTENT','')
                    # print(content_text)
                    data.append([title,content_text,category])
    return data



Scraping dilakukan perkategori
Hasil scraping perkategori kemudian dirubah menjadi dataframe dan digabungkan.
Data hasil penggabungan kemudian dilakukan random agar datanya tidak tersusun urut berdasarkan jenis kategori.
Kemudian data disimpan.






In [31]:
pendidikan = scrap_beritajatim('https://beritajatim.com/kanal/pendidikan-kesehatan/',10,'Pendidikan')

In [32]:
sport = scrap_beritajatim('https://beritajatim.com/kanal/olahraga//',4,'Sport')

In [33]:
politik = scrap_beritajatim('https://beritajatim.com/kanal/politik-pemerintahan/',6,'Politik')

In [34]:
hasil1 = pd.DataFrame(pendidikan, columns=['Judul', 'Isi', 'Kategori'])
hasil2 = pd.DataFrame(politik, columns=['Judul', 'Isi', 'Kategori'])
hasil3 = pd.DataFrame(sport, columns=['Judul', 'Isi', 'Kategori'])

In [35]:
combined_df = pd.concat([hasil1, hasil2, hasil3]).reset_index(drop=True)
combined_df

,Judul,Isi,Kategori
0,"Raih Pendanaan Matching Fund, Unisma Produksi ...",Malang (beritajatim.com)– Tim dari Universitas...,Pendidikan
1,Ratusan Guru Ngaji di Ring Satu Petrokimia Gre...,Gresik (beritajatim.com) –Ratusan guru mengaji...,Pendidikan
2,Tim PKM UB Malang Kenalkan Metode Penanaman Ve...,Malang (beritajatim.com)– Tim Program Kreativi...,Pendidikan
3,96 Persen Warga Nganjuk Terlindungi Jaminan Ke...,Nganjuk (beritajatim.com) –Pemerintah Kabupate...,Pendidikan
4,PT SBI Tuban Beri Sosialisasi Bahaya Area Tamb...,Bagikan ini,Pendidikan
5,Guru Besar ITS Gagas Olah Bijih Nikel Ramah Li...,Surabaya (beritajatim.com)– Guru Besar ITS Sur...,Pendidikan
6,"Yusron: Paham Literasi Politik, Tapi Tuna Politik","Surabaya (beritajatim.com) –Yusron Aminulloh, ...",Pendidikan
7,PENS Temukan Startup Binaan dengan Investor di...,Surabaya (beritajatim.com)– Politeknik Elektro...,Pendidikan
8,ICCWA Gelar Golden Group Indonesian Entreprene...,Surabaya (beritajatim.com)– Kamar Dagang Indon...,Pendidikan
9,Tim Spektronics ITS Surabaya Juara Dua Chem E-...,Surabaya (beritajatim.com) –Tim Spektronics IT...,Pendidikan


In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
df = combined_df.sample(frac=1).reset_index(drop=True)
df.to_csv('/content/drive/MyDrive/ppw/beritajatim.csv', index=False)

In [38]:
df

,Judul,Isi,Kategori
0,Timnas Indonesia Bakal Turunkan Amar Rayhan La...,Surabaya (beritajatim.com) –Timnas Indonesia U...,Sport
1,Tim PKM UB Malang Kenalkan Metode Penanaman Ve...,Malang (beritajatim.com)– Tim Program Kreativi...,Pendidikan
2,"Prabowo: Seribu Teman Terlalu Sedikit, Satu Mu...",Surabaya (beritajatim.com) –Calon Presiden Koa...,Politik
3,96 Persen Warga Nganjuk Terlindungi Jaminan Ke...,Nganjuk (beritajatim.com) –Pemerintah Kabupate...,Pendidikan
4,Ratusan Guru Ngaji di Ring Satu Petrokimia Gre...,Gresik (beritajatim.com) –Ratusan guru mengaji...,Pendidikan
5,PT SBI Tuban Beri Sosialisasi Bahaya Area Tamb...,Bagikan ini,Pendidikan
6,Fakhri Husaini Imbau Tak Kritik Berlebihan Tim...,Surabaya (beritajatim.com)– Performa Timnas In...,Sport
7,Tim Spektronics ITS Surabaya Juara Dua Chem E-...,Surabaya (beritajatim.com) –Tim Spektronics IT...,Pendidikan
8,"Peserta SKD CASN Kota Malang Gagal Login, Ini ...",Malang (beritajatim.com)– Ujian Seleksi Kompet...,Politik
9,Indonesia Ingin Cetak Sejarah Lewat Piala Duni...,Surabaya (beritajatim.com)– Indonesia ingin ca...,Sport


In [39]:
pd.crosstab(index=df['Kategori'], columns='count')

col_0,count
Kategori,
Pendidikan,10
Politik,6
Sport,4
